In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import nidaqmx

In [3]:
plt.style.use("dark_background")

In [4]:
from nidaqmx.stream_readers import AnalogMultiChannelReader, AnalogSingleChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter, AnalogSingleChannelWriter
from nidaqmx.constants import Edge, AcquisitionType

In [5]:
from tqdm.notebook import tqdm

In [6]:
from math import ceil

In [7]:
from ipywidgets import interact

In [8]:
%aimport brunoise.scanning_patterns

ModuleNotFoundError: No module named 'state'

In [18]:
target_framerate = 5

In [297]:
a = 1 
sample_rate = 1000000
fps = 5
n_total = sample_rate/fps
n_turn = 10
n_extra  = 200
n_y_approx = (-2*n_turn + np.sqrt(4*n_turn - 4*a*(n_extra -2*n_turn - n_total)))/(2*a)

n_y = int(round(n_y_approx))
n_x = int(round(a*n_y))

n_extra = int(round(n_total - (n_x+2*n_turn)*n_y + 2*n_turn))

raw_x, raw_y = twop.scanning_patterns.simple_scanning_pattern(n_x, n_y, n_turn, n_extra)

In [298]:
frame_duration = len(raw_x)/sample_rate

In [309]:
len(raw_x) - n_total

0.0

In [300]:
extent_x = (-3, 3)
extent_y = (-3, 3)
pos_x = raw_x*((extent_x[1]-extent_x[0])/n_x) + extent_x[0]
pos_y = raw_y*((extent_y[1]-extent_y[0])/n_y) + extent_y[0]

In [301]:
fig, ax = plt.subplots()
ax.plot(pos_x, pos_y)
ax.set_aspect(1)

<ipython-input-301-f96d005d77be>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Galvo limits

In [310]:
n_samples = len(raw_x)
time = np.arange(n_samples)/sample_rate
write_signals = np.stack([pos_x, pos_y], 0)
values_read = np.zeros((2, n_samples))


with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task:
    # Configure the channels
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1:2", min_val=-5, max_val=5)
    write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:1", min_val=-5, max_val=5)
    
    # Set the timing of both to the onboard clock so that they are synchronised
    read_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                         active_edge = Edge.RISING,
                                         sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    write_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                          active_edge = Edge.RISING,
                                          sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    
    # This is necessary to synchronise reading and wrting
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger", Edge.RISING)
    
    writer = AnalogMultiChannelWriter(write_task.out_stream)
    reader = AnalogMultiChannelReader(read_task.in_stream)
    i_save = 0

    # The first write has to be defined before the task starts
    writer.write_many_sample(write_signals)
    read_task.start()
    write_task.start()
    reader.read_many_sample(
        values_read, number_of_samples_per_channel=n_samples,
        timeout=1)

In [311]:
mystery_offset = 80

In [312]:
frame_duration

0.2

In [313]:
start_ofs = (n_x+n_turn*2)*2 + mystery_offset -n_turn
x_ovrl = values_read[0, start_ofs:start_ofs+(n_x+n_turn*2)*10].reshape(10,-1)

x_ovrl_odd = np.mean(x_ovrl[::2,:], 0)
x_ovrl_even = np.mean(x_ovrl[1::2,:], 0)

fig, ax = plt.subplots()
ax.plot(x_ovrl_odd[n_turn*2]);
ax.plot(x_ovrl_even);
ax.axvline(n_turn)
ax.axvline(n_x+n_turn)

<ipython-input-313-0f8e58905b15>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [190]:
fig, ax = plt.subplots()
ax.plot(values_read[0, :10000])
ax.plot(values_read[1,:10000])

<ipython-input-190-69e218ab6205>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [184]:
fig, ax = plt.subplots()
ax.scatter(values_read[0,:10000], values_read[1, :10000], s=0.2)
ax.set_aspect(1)

<ipython-input-184-2744298d4a38>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


# Galvo testing

In [ ]:
sample_rate = 500000.0

plane_duration = len(raw_x)/sample_rate

In [ ]:
desired_duration = 60*60*2
n_repeats = int(round(desired_duration/plane_duration))

save_every = 500
n_save_subsequent = 10

In [ ]:
n_samples = len(raw_x)
time = np.arange(n_samples)/sample_rate
write_signals = np.stack([pos_x, pos_y], 0)
read_signals = np.zeros((2, n_samples*n_saves*n_save_subsequent))
values_read = np.zeros((2, n_samples))
                       
with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task:
    # Configure the channels
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1:2", min_val=-5, max_val=5)
    write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:1", min_val=-5, max_val=5)
    
    # Set the timing of both to the onboard clock so that they are synchronised
    read_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                         active_edge = Edge.RISING,
                                         sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    write_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                          active_edge = Edge.RISING,
                                          sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    
    # This is necessary to synchronise reading and wrting
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger", Edge.RISING)
    
    writer = AnalogMultiChannelWriter(write_task.out_stream)
    reader = AnalogMultiChannelReader(read_task.in_stream)
    i_save = 0
    for i_frame in tqdm(range(n_repeats)):
        # The first write has to be defined before the task starts
        writer.write_many_sample(write_signals)
        if i_frame == 0:
            read_task.start()
            write_task.start()
        reader.read_many_sample(
            values_read, number_of_samples_per_channel=n_samples,
            timeout=1)
        if (i_frame % save_every) < n_save_subsequent:
            read_signals[:, i_save*n_samples:(i_save+1)*n_samples] = values_read
            i_save +=1 

In [ ]:
from ipywidgets import interact

In [ ]:
fig, axes = plt.subplots(1, 1, sharex=True)
plts = axes.plot(time, write_signals[0,:])

In [ ]:
fig, ax = plt.subplots()
i_sig = 1
ax.plot(time, write_signals[i_sig,:])

ln = ax.plot(time, read_signals[i_sig,0*n_samples:(0+1)*n_samples])
ax.autoscale(enable=False)
@interact
def browse_plot(i_plot:(0, read_signals.shape[1]//(n_samples)-1)=0):

    ln[0].set_ydata(read_signals[i_sig,i_plot*n_samples:(i_plot+1)*n_samples])

In [ ]:
fig, ax = plt.subplots()

@interact
def browse_plot(i_plot:(0, read_signals.shape[1]//(n_samples)-1)=0):
    ax.clear()
    ax.plot(time, write_signals[0,:])
    ax.plot(time, read_signals[0,i_plot*n_samples:(i_plot+1)*n_samples])
    ax.set_xlim(0,0.01)

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
for i in range(2):
    axes[i].plot(time, write_signals[i,:])
    axes[i].plot(time, read_signals[i,:n_samples])
    axes[i].plot(time, read_signals[i,-n_samples*2:-n_samples])

# 2p scanning

In [ ]:
from nidaqmx.constants import LineGrouping

In [ ]:
%aimport twop.objective_motor

In [ ]:
motor = twop.objective_motor.MotorControl("COM6")

In [ ]:
extent_x = (-1, 1)
extent_y = (-1, 1)

n_x = 400
n_y = 400
raw_x, raw_y = twop.scanning_patterns.simple_scanning_pattern(n_x, n_y, 10, 300, True)
pos_x = raw_x*((extent_x[1]-extent_x[0])/n_x) + extent_x[0]
pos_y = raw_y*((extent_y[1]-extent_y[0])/n_y) + extent_y[0]

n_bin = 10

n_samples_out = len(raw_x)
n_samples_in = n_samples_out*n_bin

In [ ]:
sample_rate_out = 500000.0
plane_duration = len(raw_x)/sample_rate_out

sample_rate_in = n_bin*sample_rate_out

In [ ]:
frame_duration = len(raw_x)/sample_rate_out

In [ ]:
desired_duration = 8*60*60
n_repeats = int(round(desired_duration/frame_duration))
save_every = 2000
n_saves = int(ceil(n_repeats / save_every))
n_save_subsequent = 10

In [ ]:
read_signals = np.zeros((3, n_samples_in*n_saves*n_save_subsequent))

In [ ]:
from time import sleep

Toggle shutter

In [ ]:
with nidaqmx.Task() as shutter_task:
    shutter_task.do_channels.add_do_chan("Dev1/port0/line1", line_grouping=LineGrouping.CHAN_PER_LINE)
    shutter_task.write(False, auto_start=True)
    shutter_task.write(True, auto_start=True)
    shutter_task.write(False, auto_start=True)

In [ ]:
time_in = np.arange(n_samples_in)/sample_rate_in
write_signals = np.stack([pos_x, pos_y], 0)
values_in = np.zeros((3, n_samples_in))
with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task, nidaqmx.Task() as shutter_task:
    # Configure the channels
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0:2", min_val=-5, max_val=5)
    write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:1", min_val=-10, max_val=10)
    shutter_task.do_channels.add_do_chan("Dev1/port0/line1", line_grouping=LineGrouping.CHAN_PER_LINE)
    
    # Set the timing of both to the onboard clock so that they are synchronised
    read_task.timing.cfg_samp_clk_timing(rate=sample_rate_in, source="OnboardClock",
                                         active_edge = Edge.RISING,
                                         sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples_in)
    write_task.timing.cfg_samp_clk_timing(rate=sample_rate_out, source="OnboardClock",
                                          active_edge = Edge.RISING,
                                          sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples_out)
    
    # This is necessary to synchronise reading and wrting
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger", Edge.RISING)
    
    writer = AnalogMultiChannelWriter(write_task.out_stream)
    reader = AnalogMultiChannelReader(read_task.in_stream)
    
    shutter_task.write(False, auto_start=True)
    shutter_task.write(True, auto_start=True)
    shutter_task.write(False, auto_start=True)
    sleep(0.1)
    
    i_save = 0
    for i_frame in tqdm(range(n_repeats)):
        # The first write has to be defined before the task starts
        writer.write_many_sample(write_signals)
        if i_frame == 0:
            read_task.start()
            write_task.start()
        reader.read_many_sample(
            values_in, number_of_samples_per_channel=n_samples_in,
            timeout=1)
        if i_frame % save_every == save_every-1:
            motor.move_rel(3, 0.001)
        if (i_frame % save_every) < n_save_subsequent:
            read_signals[:, i_save*n_samples_in:(i_save+1)*n_samples_in] = values_in[:,:]
            i_save += 1
    
    shutter_task.write(False, auto_start=True)
    shutter_task.write(True, auto_start=True)
    shutter_task.write(False, auto_start=True)

# Image and signal browser

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6))
i_sig = 1

image = twop.scanning_patterns.reconstruct_image_pattern(np.roll(values_in[0, :],-400), raw_x, raw_y, (n_y, n_x), n_bin)
im = ax.imshow(image, vmin=-1, vmax=0, cmap="Greys")
#ax.autoscale(enable=False)
@interact
def browse_plot(i_plot:(0, read_signals.shape[1]//(n_samples_in)-1)=0):
    im.set_data(twop.scanning_patterns.reconstruct_image_pattern(np.roll(read_signals[0, i_plot*n_samples_in:(i_plot+1)*n_samples_in],-400), raw_x, raw_y, (n_y, n_x), n_bin))

# Saving

In [ ]:
import flammkuchen as fl

In [ ]:
signals_reshaped = read_signals.reshape(3, -1, n_save_subsequent, n_samples_in//10, 10)

In [ ]:
signals_reshaped.shape

In [ ]:
fl.save("TestResults_beauty.h5", dict(read_signals=signals_reshaped, raw_x=raw_x, raw_y=raw_y, n_x=n_x, n_bin=n_bin), compression="blosc")